In [19]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import show, output_notebook
from Query import query
from time import time
from httplib2 import Http
from json import dumps
import json


output_notebook()

Loading BokehJS ...

In [20]:
#Data Wrangling

t = int(time()) - 345600

f = 'Temperature'

q = query('Garage', 'Time', '>', t, f)

#Putting Data in Pandas Dataframe
data = pd.DataFrame.from_dict(q, orient='index', columns=[f, 'Time'])
data['Time'] = pd.to_datetime(data['Time'], unit='s')
data = data.sort_values('Time')

# Formatting Pandas
data['12h'] = data['Time'].apply(lambda x: x.strftime('%d-%m-%y %p'))
data['Average'] = data.groupby(['12h']).transform('mean')
data = data.drop_duplicates('12h')
data = data.drop([f, 'Time'], axis=1)
data['12h'] = pd.to_datetime(data['12h'], format = "%d-%m-%y %p")

In [21]:
mySource = ColumnDataSource(data)
p = figure(x_axis_type='datetime', x_axis_label='Datetime', y_axis_label='Temperature(ºC)',
           title="Graph to Show Temperuture over Time",
           toolbar_location=None)

# Drawing Graph And Putting Cirles on points
p.line(x='12h', y='Average', line_width=4, color='steelblue', source=mySource)
p.circle(x='12h', y='Average', fill_color='white', line_color='orangered', size=10, line_width=3, source=mySource)

# Adding Hovertool
p.add_tools(HoverTool(
    tooltips=[
        ('Datetime', '@12h'),
        (f, '@Average' + 'ºC'),
    ],

    formatters={
        'Time': 'datetime',
    },

))

show(p)

# Explaination

The Graph above shows the temperature in the garage and how it changes over time. The temperature is an average over a 12-hour period. The graph is over a 4 days. The x-axis is the date and the y-axis is the temperature. 
>The Section of code below sends a google chat message to my phone/laptop. It sends the same data that is being displayed to the latest point of the graph above. A 12h average.

In [22]:
lastRow = data.tail(1)


with open('Cred','r') as f:
    con = json.load(f)

def main():
    url = con['Chat1']
    bot_message = {
        'text': 'The Average Temperarture in the Garage now Is: ' + str(lastRow['Average']) }

    message_headers = { 'Content-Type': 'application/json; charset=UTF-8'}

    http_obj = Http()

    response = http_obj.request(
        uri=url,
        method='POST',
        headers=message_headers,
        body=dumps(bot_message),
    )

if __name__ == '__main__':
    main()
    
print("I sent your messege!")

I sent your messege!
